In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Pheochromocytoma_and_Paraganglioma/GSE10927'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Human adrenocortical carcinomas (33), adenomas (22), and normal adrenal cortex (10), on Affymetrix HG_U133_plus_2 arrays"
!Series_summary	"Human samples of 33 adrenocortical carcinomas, 22 adrenocortical adenomas, and 10 normal adrenal cortex samples, each from a different patient, had mRNA assays performed using Affymetrix HG_U133_plus_2 arrays, with 54675 probe-sets.  We note that the same array data is in GEO series GSE33371, where we assayed the cancer samples for Beta-catenin staining or mutation, and make new comparisons based on those assays."
!Series_summary	"Keywords: disease state analysis"
!Series_overall_design	"Human samples of 33 adrenocortical carcinomas, 22 adrenocortical adenomas, and 10 normal adrenal cortex samples, each from a different patient, had mRNA assays performed using Affymetrix HG_U133_plus_2 arrays, with 54675 probe-sets."
!Series_overall_design	""
!Series_overall_design	"Two supplementary files are attached below. T

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
background_info = {
    "Series_title": "Human adrenocortical carcinomas (33), adenomas (22), and normal adrenal cortex (10), on Affymetrix HG_U133_plus_2 arrays",
    "Series_summary": ("Human samples of 33 adrenocortical carcinomas, 22 adrenocortical adenomas, "
                       "and 10 normal adrenal cortex samples, each from a different patient, had mRNA assays performed using Affymetrix HG_U133_plus_2 arrays."),
    "Series_overall_design": ("Human samples of 33 adrenocortical carcinomas, 22 adrenocortical adenomas, "
                              "and 10 normal adrenal cortex samples, each from a different patient, had mRNA assays performed using Affymetrix HG_U133_plus_2 arrays, with 54675 probe-sets.")
}

is_gene_available = True if "mRNA" in background_info["Series_summary"] else False

# Sample characteristics
sample_characteristics = {
    0: ['age: 71', 'age: 58', 'age: 44', 'age: 32', 'age: 28', 'age: 55', 'age: 78', 'age: 41', 'age: 62', 'age: 45', 'age: 25', 'age: 56', 'age: 33', 'age: 52', 'age: 54', 'age: 48', 'age: 51', 'age: 38', 'age: 64', 'age: 49', 'age: 59', 'age: 57', 'age: 35', 'age: <10', 'age: 39', 'age: 47', 'age: 60', 'age: 61', 'age: 31', 'age: 87'],
    1: ['Sex: F', 'Sex: M'],
    2: ['side of body: Left', 'side of body: Right', 'side of body: unknown', 'side of body: not applicable'],
    3: ['cinical characteristics: Adrenalectomy for metastatic lung carcinoma', "cinical characteristics: Adrenalectomy for ACA; Conn's syndrome", 'cinical characteristics: Adrenalectomy for renal cell carcinoma', 'cinical characteristics: Adrenalectomy for ACA', 'cinical characteristics: Adrenalectomy for pheochromocytoma (Multiple Endocrine Neoplasia, Type IIA)', "cinical characteristics: Adrenalectomy for multinodular cortical hyperplasia; Conn's syndrome", 'cinical characteristics: Adrenalectomy for pheochromocytoma', 'cinical characteristics: Hypertension', 'cinical characteristics: Cushing Syndrome', 'cinical characteristics: Hyperaldosteronism', 'cinical characteristics: Left adrenal mass', 'cinical characteristics: Aldosteronoma', 'cinical characteristics: Right pheochromocytoma', "cinical characteristics: Conn's syndrome", 'cinical characteristics: Abdominal pain', 'cinical characteristics: Right adrenal mass', 'cinical characteristics: Bilateral adrenal masses', 'cinical characteristics: History of Adrenocortical carcinoma', 'cinical characteristics: History of Adrenocortical carcinoma with lung, liver and bone metastises 1 year prior', 'cinical characteristics: Right adrenal tumor; Leg edema', 'cinical characteristics: Virilization and microcephaly', 'cinical characteristics: History of adrenal mass; 2 month history of abdominal pain', 'cinical characteristics: Right flank pain; Masses in right kidney, liver and both lungs', 'cinical characteristics: Cushing Syndrome; Left adrenalectomy 3 years prior for adrenocortical tumor', 'cinical characteristics: Increased testosterone', 'cinical characteristics: unknown', 'cinical characteristics: Cushing Syndrome; Right adrenalectomy 4 years prior', 'cinical characteristics: Elevated serum aldosterone and DHEA-S; History of left nephrectomy', 'cinical characteristics: Liver tumor; Severe abdominal pain', 'cinical characteristics: Right adrenal mass; elevated androgens and cortisol']
}

# Variable Availability and Data Type Conversion
trait_row = 3 if any("pheochromocytoma" in trait.lower() or "paraganglioma" in trait.lower() for trait in sample_characteristics[3]) else None
age_row = 0 if len(set(sample_characteristics[0])) > 1 else None
gender_row = 1 if "Sex: F" in sample_characteristics[1] or "Sex: M" in sample_characteristics[1] else None

# Define conversion functions
def convert_trait(value):
    parts = value.split(': ')
    val = parts[1].strip().lower()
    if 'pheochromocytoma' in val or 'paraganglioma' in val:
        return 1
    else:
        return 0

def convert_age(value):
    parts = value.split(': ')
    val = parts[1].strip()
    if val.isdigit():
        return int(val)
    elif val == '<10':
        return 5  # Consider less than 10 as 5 for simplicity
    else:
        return None

def convert_gender(value):
    parts = value.split(': ')
    val = parts[1].strip().upper()
    if val == 'F':
        return 0
    elif val == 'M':
        return 1
    else:
        return None

# Save Metadata
save_cohort_info('GSE10927', './preprocessed/Pheochromocytoma_and_Paraganglioma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Pheochromocytoma_and_Paraganglioma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Pheochromocytoma_and_Paraganglioma/trait_data/GSE10927.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM277090': [0, 71, 0], 'GSM277091': [0, 58, 0], 'GSM277092': [0, 71, 1], 'GSM277093': [0, 44, 1], 'GSM277094': [0, 32, 0], 'GSM277095': [1, 28, 1], 'GSM277096': [0, 55, 1], 'GSM277097': [0, 78, 1], 'GSM277098': [0, 41, 1], 'GSM277099': [1, 58, 1], 'GSM277100': [0, 62, 0], 'GSM277101': [0, 45, 0], 'GSM277102': [0, 28, 0], 'GSM277103': [0, 25, 0], 'GSM277104': [0, 25, 0], 'GSM277105': [0, 56, 0], 'GSM277106': [0, 33, 0], 'GSM277107': [0, 52, 1], 'GSM277108': [0, 44, 1], 'GSM277109': [1, 54, 0], 'GSM277110': [0, 71, 0], 'GSM277111': [0, 56, 0], 'GSM277112': [0, 48, 1], 'GSM277113': [0, 55, 1], 'GSM277114': [0, 51, 0], 'GSM277115': [0, 32, 0], 'GSM277116': [0, 38, 1], 'GSM277117': [0, 64, 0], 'GSM277118': [0, 49, 0], 'GSM277119': [0, 59, 1], 'GSM277120': [0, 45, 1], 'GSM277121': [0, 57, 0], 'GSM277122': [0, 41, 0], 'GSM277123': [0, 45, 0], 'GSM277124': [0, 48, 0], 'GSM277125': [0, 35, 0], 'GSM277126': [0, 71, 1], 'GSM277127': [0, 5, 0], 'GSM277128': [0, 25, 0], 'GSM277129': [0, 39, 0], 

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
if requires_gene_mapping:
    # 1. Identify keys from the gene annotation dictionary
    identifier_key = 'ID'
    gene_symbol_key = 'Gene Symbol'
    
    # 2. Get the dataframe mapping probe IDs to genes
    gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    
    # 3. Apply the gene mapping and obtain the resulting gene expression dataframe
    gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Pheochromocytoma_and_Paraganglioma/gene_data/GSE10927.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Pheochromocytoma_and_Paraganglioma')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE10927', './preprocessed/Pheochromocytoma_and_Paraganglioma/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Pheochromocytoma_and_Paraganglioma/GSE10927.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Pheochromocytoma_and_Paraganglioma', the least common label is '1.0' with 3 occurrences. This represents 4.62% of the dataset.
The distribution of the feature 'Pheochromocytoma_and_Paraganglioma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 35.0
  50% (Median): 48.0
  75%: 56.0
Min: 5.0
Max: 87.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 24 occurrences. This represents 36.92% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

